In [1]:
import time
import re
from datetime import datetime
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import json
import requests
import pandas as pd

#檢查是否為最後一頁，輸入keyword後檢索結果頁面右上角會有第幾頁，共幾頁的資料
def check_final_page(s):
    a = s.split('現在第')
    a = a[1].split('/')
    now = a[0].strip()
    total = a[1].split(' 頁')[0].strip()
    if(now=='25'): return True
    return (now==total)


main_driver = webdriver.Chrome('C:\\Users\\user\\Desktop\\Conda\\chromedriver')  # 注意你們放CHROMEDRIVER的位置
main_driver.implicitly_wait(10)
main_driver.get('https://law.judicial.gov.tw/FJUD/defaulte.aspx')

#簡單檢所的頁面中，輸入關鍵字，這邊你們可以放入洗錢嫌疑程度的關鍵字
keyword_input_ele = main_driver.find_element_by_xpath('//*[@id="txtKW"]')
keyword_input_ele.send_keys("背信")

btn_ele = keyword_input_ele.find_element_by_xpath('//*[@id="btnSimpleQry"]')
btn_ele.click()
#這邊很重要，要切換到iframe內
main_driver.switch_to.frame("iframe-data")
case_link_list=[]

while True:
    
    ##取的一頁中的所有檢索案件
    case_list = main_driver.find_elements_by_id('hlTitle')
    for idx, case in enumerate(case_list):
        case_link_list.append(case.get_attribute('href'))#將每個案件的檢索結果頁面連結放在case_link_list中
    #注意因為網頁最多只會提供500比檢索資料，因此case_link_list長度最多500
    #如果是最後一頁，就跳出
    if(not check_final_page(main_driver.find_element_by_xpath('//*[@id="plPager"]/span[1]').text)):
        next_page_btn_ele = main_driver.find_element_by_id('hlNext')
        next_page_btn_ele.click()  
    else: break
main_driver.close()

In [12]:
#下面是其中一頁的判決書範例(我自己編的範例，非正確判決書內容，但格式都長這樣)
"""
裁判字號：
臺中簡易庭 106 年中勞小字第 19 號民事判決
裁判日期：
民國 106 年 12 月 15 日
裁判案由：
給付資遣費等
臺灣臺中地方法院民事判決　　　　　 106年度中勞小字第19號
原　　　告　童舶裕
被　　　告　九福國際人力仲介有限公司
　　　　　　謝○○
法定代理人　謝文卿
訴訟代理人　蔡宛陵
　　　　　　林合郁
　　　　　　張魁育
上列當事人間請求給付資遣費等事件，於民國106年11月28日言
詞辯論終結，本院判決如下：
    主      文
被告應給付原告新臺幣肆萬伍仟參佰參拾參元，及自民國一百零
六年三月十日起至清償日止，按週年利率百分之五計算之利息。
原告其餘之訴駁回。
"""

import pandas as pd
df = pd.DataFrame(columns=['人名','正文'])
nonsense_char_list=['○','.','律師','○','○○']
start_targets_list = ["被　　　告","即被告　","上訴人即被告　"]

def Check_Noncense(line_string):#注意在第12行，謝○○的人名這樣的就可以用此方法去掉
    line_string = line_string.strip()
    for nonsense_char in nonsense_char_list:
        if(len(line_string)<4 and line_string.find(nonsense_char)!= -1): return True#大於四個字的有可能會有相關資訊
    return False    

#注意在第11行，找到有"被　　　告"的關鍵字之後就開始搜尋這個項目裡的名子，直到
#第12行的「謝○○」為止
def Check_Start_Target_Title(line_string): 
    output=''
    for start_target in start_targets_list:
        if(line_string.find(start_target)!= -1):            
            output = line_string.replace(start_target,"")
            return output, True
    return output, False

#做把各篇判決書內的被告取出名子的方法
def Get_Name_List(context):
    start_checking_mode = False
    temp = pd.DataFrame(columns=['人名','正文'])
    #先把判決書的每行分開
    lines = context.split('\n')
    #接著對判決書內的每行做處理   
    for line in lines:
        if(start_checking_mode): 
            #步驟3，接著找下一行是否有同樣屬於被告等類似的title範圍內的名子，這裡參考第12行，前面會有很長的空格
            if(line.find("　　　　　")==0): #步驟4，如果有且Check_Noncense發現裡面不是存在無法辨識的名子，則放入df中
                if(Check_Noncense(line) == False): temp = temp.append({'人名':line.strip(),'正文':context}, ignore_index=True)            
            else: start_checking_mode = False #步驟5，如果在步驟4沒發現，則繼續望下一行找(跳回地62行)
        else:
            #步驟1，先確定是否有被告等類似的title出現，參考第11行範例，如果有出現，則start_checking_mode=true
            #如果沒有，則繼續找看看下一行否有被告等類似的title出現
            start_name, start_checking_mode  =  Check_Start_Target_Title(line)
            if(start_checking_mode) : 
                #步驟2，如果找到，則先把該行名子取出來，這裡會取出第11行的"九福國際人力仲介有限公司"
                if(Check_Noncense(start_name) == False): temp = temp.append({'人名':start_name,'正文':context}, ignore_index=True)
    return temp


main_driver = webdriver.Chrome('C:\\Users\\user\\Desktop\\Conda\\chromedriver')  # 注意你們放CHROMEDRIVER的位置
main_driver.implicitly_wait(10)
for case_link in case_link_list:
    if(case_link==None): continue 
    #----------------這邊從每篇判決書文字中取得我們要的人名------------------#
    main_driver.get(case_link)
    case_text = main_driver.find_element_by_class_name('text-pre').text
    df = df.append(Get_Name_List(case_text))
    time.sleep(1)
    #-------------------------------------------------------------------#
main_driver.close()
#變成dataframe方便之後處理，接下來就是上傳到資料庫啦~
df

,人名,正文
0,財團法人法律扶助基金會,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
1,佘家玲,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
2,蔡雲卿,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
3,李文平,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
0,群豐科技股份有限公司,臺灣苗栗地方法院民事簡易判決 108年度苗勞簡字第2號\n原 告 彭珮瑩 \n訴訟...
0,譚小婷,臺灣臺北地方法院簡易民事判決 107年度北簡字第10606號\n原 告 太平洋建設股...
0,基隆市山海觀公寓大廈管理委員會,臺灣基隆地方法院民事判決\n 106年度訴字第513號\...
0,王秀明,臺灣新竹地方法院刑事裁定 107年度聲判字第40號\n聲 請 人\n即 告 訴...
0,彭玉熾,臺灣新竹地方法院刑事判決 107年度易字第854號\n公 訴 人 臺灣新竹地方檢...
0,楊榮欽,臺灣新北地方法院板橋簡易庭民事簡易判決\n 108年度板簡...


In [13]:
df = df.reset_index(drop=True)
df

,人名,正文
0,財團法人法律扶助基金會,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
1,佘家玲,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
2,蔡雲卿,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
3,李文平,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
4,群豐科技股份有限公司,臺灣苗栗地方法院民事簡易判決 108年度苗勞簡字第2號\n原 告 彭珮瑩 \n訴訟...
5,譚小婷,臺灣臺北地方法院簡易民事判決 107年度北簡字第10606號\n原 告 太平洋建設股...
6,基隆市山海觀公寓大廈管理委員會,臺灣基隆地方法院民事判決\n 106年度訴字第513號\...
7,王秀明,臺灣新竹地方法院刑事裁定 107年度聲判字第40號\n聲 請 人\n即 告 訴...
8,彭玉熾,臺灣新竹地方法院刑事判決 107年度易字第854號\n公 訴 人 臺灣新竹地方檢...
9,楊榮欽,臺灣新北地方法院板橋簡易庭民事簡易判決\n 108年度板簡...


In [47]:
org_filter_list = ['公司','委員會','基金會']
df_org = pd.DataFrame(columns=['法人名稱','判決正文','犯罪類別'])
df_people = pd.DataFrame(columns=['人名','判決正文','犯罪類別'])


df_org = df[df['人名'].str.contains('|'.join(org_filter_list))]
df_people = df[~df['人名'].str.contains('|'.join(org_filter_list))]

df_people.head()

,人名,正文
1,佘家玲,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
2,蔡雲卿,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
3,李文平,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
5,譚小婷,臺灣臺北地方法院簡易民事判決 107年度北簡字第10606號\n原 告 太平洋建設股...
7,王秀明,臺灣新竹地方法院刑事裁定 107年度聲判字第40號\n聲 請 人\n即 告 訴...


In [48]:
df_org.head()

,人名,正文
0,財團法人法律扶助基金會,臺灣新竹地方法院民事判決 108年度國字第1號\n原 告 周惠竹 \n被 ...
4,群豐科技股份有限公司,臺灣苗栗地方法院民事簡易判決 108年度苗勞簡字第2號\n原 告 彭珮瑩 \n訴訟...
6,基隆市山海觀公寓大廈管理委員會,臺灣基隆地方法院民事判決\n 106年度訴字第513號\...
32,煜興科技有限公司,臺灣臺中地方法院民事裁定 108年度中小字第1593號\n原 告 樂衍有限公司...
40,大庭新城公寓大廈管理委員會,臺灣新北地方法院板橋簡易庭簡易民事判決\n 107年度板建簡字第...
